## script para generar tablas de mutaciones para base de datos

In [322]:
import pandas as pd
import numpy as np
from Bio.SeqUtils import seq3

In [323]:
clinvar = pd.read_csv('datasets/clinvar_box1_mutations.csv.gz', compression='gzip')
disgenet = pd.read_csv('datasets/disgenet_box1_mutations.csv.gz', compression='gzip')
uniprot = pd.read_csv('datasets/uniprot_box1_variants.csv')

In [324]:
uniprot = pd.read_csv('datasets/uniprot_box1_variants.csv')

In [377]:
box1 = pd.read_csv('box1_proteins.csv')

In [325]:
clinvar = clinvar.rename(columns={'snpid': 'snp_id', 'geneid': 'gene_id', 'genesymbol': 'gene_name', 'start': 'start_genomic', 'stop': 'end_genomic', 'from': 'from_aa', 'to': 'to_aa'})

In [326]:
clinvar_subset = clinvar[['uniprot', 'hgnc_id', 'gene_id',
       'gene_name', 'snp_id', 'chromosome', 'start_genomic', 'end_genomic', 'cambio', 'consequence',
       'start_aa', 'end_aa', 'from_aa', 'to_aa', 'source']]
clinvar_subset

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,start_genomic,end_genomic,cambio,consequence,start_aa,end_aa,from_aa,to_aa,source
0,O95613,HGNC:16068,5116,PCNT,587784306,21,46334558,46334596,His156_Gln168del,deletion,156,168.0,HisGln,NaN,clinvar
1,O95613,HGNC:16068,5116,PCNT,1555945478,21,46334536,46334613,Val148_Thr173del,deletion,148,173.0,ValThr,NaN,clinvar
2,O95613,HGNC:16068,5116,PCNT,562568796,21,46435981,46435983,Lys2944del,deletion,2944,NaN,Lys,NaN,clinvar
3,O95613,HGNC:16068,5116,PCNT,759470656,21,46385936,46385938,Leu1141del,deletion,1141,NaN,Leu,NaN,clinvar
4,P46100,HGNC:886,546,ATRX,398123423,X,77652292,77652294,Glu1464del,deletion,1464,NaN,Glu,NaN,clinvar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23121,Q15233,HGNC:7871,4841,NONO,1057524408,X,71290740,71290740,Gln35Ter,nonsense,35,NaN,Gln,Ter,clinvar
23122,Q15233,HGNC:7871,4841,NONO,1085307870,X,71297442,71297442,Arg337Ter,nonsense,337,NaN,Arg,Ter,clinvar
23123,Q15233,HGNC:7871,4841,NONO,1555949215,X,71291841,71291841,Arg73Ter,nonsense,73,NaN,Arg,Ter,clinvar
23124,Q15233,HGNC:7871,4841,NONO,1602387702,X,71294428,71294428,Arg184Ter,nonsense,184,NaN,Arg,Ter,clinvar


In [327]:
len(clinvar.snp_id.unique())

16367

In [328]:
clinvar.snp_id.value_counts() # hay snps repetidos

-1             5123
 113070757       12
 746430067        5
 782576893        5
 72550890         5
               ... 
 150168649        1
 1553265739       1
 767837521        1
 1554138189       1
 1554581580       1
Name: snp_id, Length: 16367, dtype: int64

In [329]:
a = clinvar[clinvar.snp_id == 746430067]
print(a.cambio)

268    Pro187_Pro189del
270    Pro188_Pro189del
271           Pro189del
591           Pro189dup
592    Pro187_Pro189dup
Name: cambio, dtype: object


In [330]:
clinvar.columns

Index(['uniprot', 'organism', 'mlo', 'hgnc_id', 'length', 'sequence',
       'gene_id', 'gene_name', 'snp_id', 'alleleid', 'chromosomeaccession',
       'chromosome', 'start_genomic', 'end_genomic', 'type', 'name', 'origin',
       'phenotypeids', 'phenotypelist', 'otherids', 'nuccore_id', 'cambio',
       'consequence', 'start_aa', 'end_aa', 'from_aa', 'to_aa', 'source'],
      dtype='object')

In [331]:
disgenet.columns

Index(['uniprot', 'organism', 'mlo', 'hgnc_id', 'length', 'sequence', 'snp_id',
       'chromosome', 'position', 'dsi', 'dpi', 'diseaseid', 'diseasename',
       'diseasetype', 'diseaseclass', 'diseasesemantictype', 'score', 'ei',
       'yearinitial', 'yearfinal', 'nofpmids', 'source', 'gene_name',
       'gene_id', 'consequence', 'alleles', 'class', 'af_exome',
       'alt_ref_exome', 'af_genome', 'alt_ref_genome', 'source_id'],
      dtype='object')

## todas las cols a comparar en mismo formato y tipo de dato

In [332]:
# en la col snp de disgenet saco el rs y paso a int para comparar con clinvar
disgenet.snp_id = disgenet.snp_id.str.strip('rs').apply(int)
disgenet.snp_id

0         104893751
1          35233638
2          35233638
3         119479061
4         119479062
            ...    
11544      79546472
11545    1114167441
11546     869025343
11547     869025345
11548     876661316
Name: snp_id, Length: 11549, dtype: int64

In [333]:
# eliminar el rs y paso a int
uniprot.snp_id = uniprot.snp_id.str.strip('rs')
uniprot.snp_id = uniprot.snp_id.replace('-', -1) # cambio los "-" por -1 asi toda la col sea INT
uniprot.snp_id = uniprot.snp_id.apply(int)
uniprot.snp_id

0       104893751
1        56053615
2         1052133
3         1805373
4         1801128
          ...    
4089    104894163
4090     11567901
4091           -1
4092           -1
4093     17853616
Name: snp_id, Length: 4094, dtype: int64

## Busco todas las entradas de disgenet que esten en clinvar

In [334]:
disgenet.source.value_counts()

CLINVAR            7720
GWASCAT            1774
GWASDB              887
UNIPROT             513
CLINVAR;UNIPROT     498
GWASCAT;GWASDB      156
CLINVAR;GWASDB        1
Name: source, dtype: int64

In [335]:
disgenet_subset_clinvar = disgenet[(disgenet.source == 'CLINVAR') | (disgenet.source == 'CLINVAR;UNIPROT') | (disgenet.source == 'CLINVAR;GWASDB')]
len(disgenet_subset_clinvar)

8219

In [336]:
len(disgenet_subset_clinvar.snp_id.unique())

3926

In [337]:
disgenet_subset_clinvar.snp_id.value_counts() # hay snps repetidos xq tienen un disease por fila

1554700718    59
1554333853    54
1569355102    51
11540652      42
28934576      39
              ..
878854068      1
772005832      1
121912440      1
1569558474     1
878854136      1
Name: snp_id, Length: 3926, dtype: int64

## subseteo disgenet sin tener en cuenta los diseases asi me quedo con los snps unicos

In [338]:
clinvar_subset.columns

Index(['uniprot', 'hgnc_id', 'gene_id', 'gene_name', 'snp_id', 'chromosome',
       'start_genomic', 'end_genomic', 'cambio', 'consequence', 'start_aa',
       'end_aa', 'from_aa', 'to_aa', 'source'],
      dtype='object')

In [339]:
disgenet.columns

Index(['uniprot', 'organism', 'mlo', 'hgnc_id', 'length', 'sequence', 'snp_id',
       'chromosome', 'position', 'dsi', 'dpi', 'diseaseid', 'diseasename',
       'diseasetype', 'diseaseclass', 'diseasesemantictype', 'score', 'ei',
       'yearinitial', 'yearfinal', 'nofpmids', 'source', 'gene_name',
       'gene_id', 'consequence', 'alleles', 'class', 'af_exome',
       'alt_ref_exome', 'af_genome', 'alt_ref_genome', 'source_id'],
      dtype='object')

In [340]:
disgenet_subset_clinvar = disgenet_subset_clinvar[['uniprot', 'hgnc_id','gene_id', 'gene_name', 'snp_id', 'chromosome', 'position', 'consequence', 'source']]
disgenet_subset_clinvar

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,position,consequence,source
0,O15527,HGNC:8125,4968,OGG1,104893751,3,9750423.0,missense variant,CLINVAR
3,O95613,HGNC:16068,5116,PCNT,119479061,21,46346146.0,stop gained,CLINVAR
4,O95613,HGNC:16068,5116,PCNT,119479062,21,46411840.0,stop gained,CLINVAR
5,O95613,HGNC:16068,5116,PCNT,119479063,21,46367083.0,stop gained,CLINVAR
6,O95613,HGNC:16068,5116,PCNT,119479064,21,46435904.0,stop gained,CLINVAR
...,...,...,...,...,...,...,...,...,...
11512,P23771,HGNC:4172,2625,GATA3,387906621,10,8069573.0,missense variant,CLINVAR
11545,Q15233,HGNC:7871,4841,NONO,1114167441,X,71298508.0,missense variant,CLINVAR
11546,Q15233,HGNC:7871,4841,NONO,869025343,X,71297938.0,splice region variant,CLINVAR
11547,Q15233,HGNC:7871,4841,NONO,869025345,X,71297900.0,stop gained,CLINVAR


In [341]:
disgenet_subset_clinvar.consequence.value_counts()

missense variant                      3759
frameshift variant                    1827
stop gained                           1467
splice donor variant                   416
splice acceptor variant                361
splice region variant                  153
inframe deletion                        71
non coding transcript exon variant      63
intron variant                          33
start lost                              14
synonymous variant                      12
inframe insertion                       10
5 prime UTR variant                     10
3 prime UTR variant                      7
stop lost                                5
protein altering variant                 5
coding sequence variant                  4
upstream gene variant                    2
Name: consequence, dtype: int64

In [342]:
disgenet_subset_clinvar = disgenet_subset_clinvar.drop_duplicates(subset=['snp_id']) # 3926 snps unicos en disgenet con source de clinvar

In [343]:
disgenet_subset_clinvar

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,position,consequence,source
0,O15527,HGNC:8125,4968,OGG1,104893751,3,9750423.0,missense variant,CLINVAR
3,O95613,HGNC:16068,5116,PCNT,119479061,21,46346146.0,stop gained,CLINVAR
4,O95613,HGNC:16068,5116,PCNT,119479062,21,46411840.0,stop gained,CLINVAR
5,O95613,HGNC:16068,5116,PCNT,119479063,21,46367083.0,stop gained,CLINVAR
6,O95613,HGNC:16068,5116,PCNT,119479064,21,46435904.0,stop gained,CLINVAR
...,...,...,...,...,...,...,...,...,...
11512,P23771,HGNC:4172,2625,GATA3,387906621,10,8069573.0,missense variant,CLINVAR
11545,Q15233,HGNC:7871,4841,NONO,1114167441,X,71298508.0,missense variant,CLINVAR
11546,Q15233,HGNC:7871,4841,NONO,869025343,X,71297938.0,splice region variant,CLINVAR
11547,Q15233,HGNC:7871,4841,NONO,869025345,X,71297900.0,stop gained,CLINVAR


# Disgenet con Clinvar
## Buscar los snps de disgenet con source "clinvar" en el dataset de clinvar

In [344]:
# con isin
disgenet_en_clinvar = clinvar_subset[clinvar_subset.snp_id.isin(disgenet_subset_clinvar.snp_id)] # estoy subseteando clinvar, no fusionando las tablas
disgenet_en_clinvar

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,start_genomic,end_genomic,cambio,consequence,start_aa,end_aa,from_aa,to_aa,source
19,Q96FC9,HGNC:2736,1663,DDX11,730880280,12,31103727,31103729,Lys897del,deletion,897,NaN,Lys,NaN,clinvar
29,P18583,HGNC:11183,6651,SON,769691894,21,33553375,33553398,Leu1384_Val1391del,deletion,1384,1391.0,LeuVal,NaN,clinvar
30,P18583,HGNC:11183,6651,SON,1569058041,21,33554229,33554243,Asp1667_Asn1671del,deletion,1667,1671.0,AspAsn,NaN,clinvar
76,P04637,HGNC:11998,7157,TP53,587782490,17,7676030,7676041,Arg110_Phe113del,deletion,110,113.0,ArgPhe,NaN,clinvar
79,P04637,HGNC:11998,7157,TP53,876659260,17,7674227,7674247,Asn239_Gly245del,deletion,239,245.0,AsnGly,NaN,clinvar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23116,P04156,HGNC:9449,5621,PRNP,17852079,20,4699899,4699899,Gln227Ter,nonsense,227,NaN,Gln,Ter,clinvar
23117,P04156,HGNC:9449,5621,PRNP,1555782101,20,4699709,4699709,Tyr163Ter,nonsense,163,NaN,Tyr,Ter,clinvar
23118,P23771,HGNC:4172,2625,GATA3,104894162,10,8064043,8064043,Arg277Ter,nonsense,277,NaN,Arg,Ter,clinvar
23119,P23771,HGNC:4172,2625,GATA3,104894164,10,8073787,8073787,Arg367Ter,nonsense,367,NaN,Arg,Ter,clinvar


In [345]:
disgenet_en_clinvar.snp_id.value_counts()

587776585     4
1057519744    4
63750264      3
587778720     3
143985153     3
             ..
1161032867    1
398122893     1
1555619402    1
786203589     1
1555611570    1
Name: snp_id, Length: 3180, dtype: int64

In [346]:
disgenet_en_clinvar[disgenet_en_clinvar.snp_id == 1057519744]

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,start_genomic,end_genomic,cambio,consequence,start_aa,end_aa,from_aa,to_aa,source
1148,P06748,HGNC:7910,4869,NPM1,1057519744,5,171410542,171410543,Trp288fs,frameshift,288,NaN,Trp,NaN,clinvar
1149,P06748,HGNC:7910,4869,NPM1,1057519744,5,171410542,171410543,Trp288fs,frameshift,288,NaN,Trp,NaN,clinvar
1150,P06748,HGNC:7910,4869,NPM1,1057519744,5,171410542,171410543,Trp288fs,frameshift,288,NaN,Trp,NaN,clinvar
1151,P06748,HGNC:7910,4869,NPM1,1057519744,5,171410542,171410543,Trp288fs,frameshift,288,NaN,Trp,NaN,clinvar


In [347]:
len(disgenet_en_clinvar.snp_id.unique())

3180

In [348]:
#clinvar_subset.merge(disgenet_subset_clinvar, left_on=['snp_id'], right_on=['snp_id']) # dan lo mismo, pero aca estoy combinando las tablas

In [349]:
keep = ~disgenet_subset_clinvar.snp_id.isin(disgenet_en_clinvar.snp_id) # get only what is unique (or not in) disgenet_en_clinvar
entradas_disgenet_eliminar = disgenet_subset_clinvar[keep]  
entradas_disgenet_eliminar # si bien estan en clinvar, al buscar estos snps se ve que no tienen efecto en proteina (Protein change)
                           # por lo tanto hay que eliminarlas del dataset de disgenet

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,position,consequence,source
12,O95613,HGNC:16068,5116,PCNT,1555954786,21,46353985.0,splice acceptor variant,CLINVAR
14,O95613,HGNC:16068,5116,PCNT,1555959737,21,46363457.0,splice acceptor variant,CLINVAR
18,O95613,HGNC:16068,5116,PCNT,1569178877,21,46347513.0,splice donor variant,CLINVAR
33,O95613,HGNC:16068,5116,PCNT,755084205,21,46388741.0,splice acceptor variant,CLINVAR
52,P38159,HGNC:9910,27316,RBMX,181515589,X,136876663.0,splice region variant,CLINVAR
...,...,...,...,...,...,...,...,...,...
11338,P03372,HGNC:3467,2099,ESR1,1562787582,6,152099207.0,3 prime UTR variant,CLINVAR
11490,P23771,HGNC:4172,2625,GATA3,112417755,10,8073738.0,splice acceptor variant,CLINVAR
11511,P23771,HGNC:4172,2625,GATA3,387906551,10,8064140.0,splice donor variant,CLINVAR
11546,Q15233,HGNC:7871,4841,NONO,869025343,X,71297938.0,splice region variant,CLINVAR


In [407]:
entradas_disgenet_eliminar.consequence.value_counts()

splice donor variant                  239
splice acceptor variant               196
frameshift variant                    146
splice region variant                  66
missense variant                       29
intron variant                         22
stop gained                            15
synonymous variant                      7
3 prime UTR variant                     6
5 prime UTR variant                     5
inframe deletion                        4
non coding transcript exon variant      4
coding sequence variant                 3
upstream gene variant                   2
inframe insertion                       1
protein altering variant                1
Name: consequence, dtype: int64

In [406]:
disgenet.columns

Index(['uniprot', 'organism', 'mlo', 'hgnc_id', 'length', 'sequence', 'snp_id',
       'chromosome', 'position', 'dsi', 'dpi', 'diseaseid', 'diseasename',
       'diseasetype', 'diseaseclass', 'diseasesemantictype', 'score', 'ei',
       'yearinitial', 'yearfinal', 'nofpmids', 'source', 'gene_name',
       'gene_id', 'consequence', 'alleles', 'class', 'af_exome',
       'alt_ref_exome', 'af_genome', 'alt_ref_genome', 'source_id'],
      dtype='object')

In [405]:
disgenet[disgenet.snp_id == 1555954786]

,uniprot,organism,mlo,hgnc_id,length,sequence,snp_id,chromosome,position,dsi,...,gene_name,gene_id,consequence,alleles,class,af_exome,alt_ref_exome,af_genome,alt_ref_genome,source_id
12,O95613,Homo sapiens,centrosome/spindle pole body,HGNC:16068,3336,MEVEQEQRRRKVEAGRTKLAHFRQRKTKGDSSHSEKKTAKRKGSAV...,1555954786,21,46353985.0,1.0,...,PCNT,5116,splice acceptor variant,A/G,snv,NaN,NaN,NaN,NaN,VEP


In [350]:
# da lo mismo de esta forma
cond = disgenet_subset_clinvar.snp_id.isin(disgenet_en_clivar.snp_id)                           # es un array de bool
disgenet_subset_clinvar.drop(disgenet_subset_clinvar[cond].index)   # drop esas filas


,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,position,consequence,source
12,O95613,HGNC:16068,5116,PCNT,1555954786,21,46353985.0,splice acceptor variant,CLINVAR
14,O95613,HGNC:16068,5116,PCNT,1555959737,21,46363457.0,splice acceptor variant,CLINVAR
18,O95613,HGNC:16068,5116,PCNT,1569178877,21,46347513.0,splice donor variant,CLINVAR
33,O95613,HGNC:16068,5116,PCNT,755084205,21,46388741.0,splice acceptor variant,CLINVAR
52,P38159,HGNC:9910,27316,RBMX,181515589,X,136876663.0,splice region variant,CLINVAR
...,...,...,...,...,...,...,...,...,...
11338,P03372,HGNC:3467,2099,ESR1,1562787582,6,152099207.0,3 prime UTR variant,CLINVAR
11490,P23771,HGNC:4172,2625,GATA3,112417755,10,8073738.0,splice acceptor variant,CLINVAR
11511,P23771,HGNC:4172,2625,GATA3,387906551,10,8064140.0,splice donor variant,CLINVAR
11546,Q15233,HGNC:7871,4841,NONO,869025343,X,71297938.0,splice region variant,CLINVAR


# Uniprot con Clinvar

In [351]:
len(uniprot.snp_id.unique())

2554

In [352]:
uniprot_subset = uniprot[~(uniprot.snp_id == -1)] # snps que no son -1
uniprot_subset = uniprot_subset[['uniprot', 'hgnc_id', 'gene_name', 'snp_id', 'change', 'aa1',	'pos', 'aa2']]
uniprot_subset = uniprot_subset.drop_duplicates(subset=['snp_id']) # elimino los snp repetidos (xq tienen mas de un disease)
uniprot_subset

,uniprot,hgnc_id,gene_name,snp_id,change,aa1,pos,aa2
0,O15527,HGNC:8125,OGG1,104893751,p.Arg46Gln,R,46,Q
1,O15527,HGNC:8125,OGG1,56053615,p.Arg154His,R,154,H
2,O15527,HGNC:8125,OGG1,1052133,p.Ser326Cys,S,326,C
3,O15527,HGNC:8125,OGG1,1805373,p.Arg229Gln,R,229,Q
4,O15527,HGNC:8125,OGG1,1801128,p.Ser320Thr,S,320,T
...,...,...,...,...,...,...,...,...
4087,P04156,HGNC:9449,PRNP,74315410,p.Gly131Val,G,131,V
4088,P04156,HGNC:9449,PRNP,267606980,p.Gly127Val,G,127,V
4089,P23771,HGNC:4172,GATA3,104894163,p.Trp274Arg,W,274,R
4090,P23771,HGNC:4172,GATA3,11567901,p.Gly242Ser,G,242,S


In [353]:
# con isin
uniprot_subset_clinvar = clinvar_subset[clinvar_subset.snp_id.isin(uniprot_subset.snp_id)] # estoy subseteando clinvar, no fusionando las tablas
uniprot_subset_clinvar


,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,start_genomic,end_genomic,cambio,consequence,start_aa,end_aa,from_aa,to_aa,source
3552,O15527,HGNC:8125,4968,OGG1,104893751,3,9750423,9750423,Arg46Gln,missense,46,NaN,Arg,Gln,clinvar
3553,O43663,HGNC:9341,9055,PRC1,12911192,15,90970444,90970444,Tyr511Cys,missense,511,NaN,Tyr,Cys,clinvar
3555,O95613,HGNC:16068,5116,PCNT,35940413,21,46411595,46411595,Asn1841Ser,missense,1841,NaN,Asn,Ser,clinvar
3557,O95613,HGNC:16068,5116,PCNT,34268261,21,46411931,46411931,Arg1953His,missense,1953,NaN,Arg,His,clinvar
3559,O95613,HGNC:16068,5116,PCNT,2839256,21,46428545,46428545,Ala2549Thr,missense,2549,NaN,Ala,Thr,clinvar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22635,P60484,HGNC:9588,5728,PTEN,121909224,10,87933147,87933147,Arg130Ter,nonsense,130,NaN,Arg,Ter,clinvar
22642,P60484,HGNC:9588,5728,PTEN,587782603,10,87952118,87952118,Gly165Ter,nonsense,165,NaN,Gly,Ter,clinvar
22681,P60484,HGNC:9588,5728,PTEN,562015640,10,87960957,87960957,Lys289Ter,nonsense,289,NaN,Lys,Ter,clinvar
22710,P60484,HGNC:9588,5728,PTEN,786202398,10,87925518,87925518,Leu57Ter,nonsense,57,NaN,Leu,Ter,clinvar


In [354]:
#clinvar_subset.merge(uniprot_subset, on='snp_id') # da lo mismo,
#clinvar_subset.merge(uniprot_subset, left_on=['snp_id'], right_on=['snp_id']) # da lo mismo

In [355]:
len(uniprot_subset_clinvar.snp_id.unique()) # 1567 snps unicos de uniprot en clinvar, por lo tanto no hay que agregarlos

1567

In [356]:
# esta diferencia son los snps de Uniprot que NO estan en clinvar
# entonces hay que agregarlos a la tabla de mutaciones
cond = ~uniprot_subset.snp_id.isin(uniprot_subset_clinvar.snp_id)  # get only what is unique (or not in) disgenet_en_clinvar
entradas_solo_uniprot = uniprot_subset[cond]  
entradas_solo_uniprot

,uniprot,hgnc_id,gene_name,snp_id,change,aa1,pos,aa2
1,O15527,HGNC:8125,OGG1,56053615,p.Arg154His,R,154,H
2,O15527,HGNC:8125,OGG1,1052133,p.Ser326Cys,S,326,C
3,O15527,HGNC:8125,OGG1,1805373,p.Arg229Gln,R,229,Q
4,O15527,HGNC:8125,OGG1,1801128,p.Ser320Thr,S,320,T
5,O15527,HGNC:8125,OGG1,3219012,p.Ala288Val,A,288,V
...,...,...,...,...,...,...,...,...
4082,P04156,HGNC:9449,PRNP,761807915,p.Asp202Asn,D,202,N
4083,P04156,HGNC:9449,PRNP,776593792,p.Val203Ile,V,203,I
4085,P04156,HGNC:9449,PRNP,751882709,p.Gln212Pro,Q,212,P
4090,P23771,HGNC:4172,GATA3,11567901,p.Gly242Ser,G,242,S


In [357]:
uniprot[uniprot.snp_id.isin(entradas_solo_uniprot.snp_id)] # estas son las entradas en la tabla original de uniprot

,uniprot,organism,mlo,hgnc_id,length,sequence,gene_name,ft_id,change,category,snp_id,mim,disease,aa1,pos,aa2
1,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...,OGG1,VAR_009520,p.Arg154His,LB/B,56053615,NaN,-,R,154,H
2,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...,OGG1,VAR_009521,p.Ser326Cys,LB/B,1052133,NaN,-,S,326,C
3,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...,OGG1,VAR_014487,p.Arg229Gln,LB/B,1805373,NaN,-,R,229,Q
4,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...,OGG1,VAR_014488,p.Ser320Thr,LB/B,1801128,NaN,-,S,320,T
5,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...,OGG1,VAR_018890,p.Ala288Val,LB/B,3219012,NaN,-,A,288,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4082,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,PRNP,VAR_008750,p.Asp202Asn,LP/P,761807915,MIM:137440,Gerstmann-Straussler disease (GSD),D,202,N
4083,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,PRNP,VAR_008751,p.Val203Ile,US,776593792,MIM:123400,Creutzfeldt-Jakob disease (CJD),V,203,I
4085,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,PRNP,VAR_008753,p.Gln212Pro,LP/P,751882709,MIM:137440,Gerstmann-Straussler disease (GSD),Q,212,P
4090,P23771,Homo sapiens,enhanceosome,HGNC:4172,443,MEVTADQPRWVSHHHPAVLNGQHPDTHHPGLSHSYMDAAQYPLPEE...,GATA3,VAR_019202,p.Gly242Ser,LB/B,11567901,NaN,-,G,242,S


# Generar tabla de mutaciones

In [382]:
mutations = clinvar[['snp_id', 'chromosome', 'start_genomic', 'end_genomic', 'start_aa', 'end_aa', 'from_aa', 'to_aa', 'consequence', 'source']]

## agrego uniprot

In [358]:
# doy formato para la db
entradas_solo_uniprot['consequence'] = 'missense' # todo el dataset humsavar.txt de uniprot son missense variants
entradas_solo_uniprot['source'] = 'uniprot'
entradas_solo_uniprot = entradas_solo_uniprot.rename(columns={'pos': 'start_aa', 'aa1': 'from_aa', 'aa2': 'to_aa', 'change': 'cambio'})
entradas_solo_uniprot.cambio = entradas_solo_uniprot.cambio.str.lstrip('p.')

<ipython-input-358-7a4c91be6a79>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entradas_solo_uniprot['consequence'] = 'missense' # todo el dataset humsavar de uniprot son missense variants
<ipython-input-358-7a4c91be6a79>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entradas_solo_uniprot['source'] = 'uniprot'


In [359]:
# pasar aa en tres letras
entradas_solo_uniprot['from_aa'] = entradas_solo_uniprot['from_aa'].map(lambda x: seq3(x))
entradas_solo_uniprot['to_aa'] = entradas_solo_uniprot['to_aa'].map(lambda x: seq3(x))
entradas_solo_uniprot

,uniprot,hgnc_id,gene_name,snp_id,cambio,from_aa,start_aa,to_aa,consequence,source
1,O15527,HGNC:8125,OGG1,56053615,Arg154His,Arg,154,His,missense,uniprot
2,O15527,HGNC:8125,OGG1,1052133,Ser326Cys,Ser,326,Cys,missense,uniprot
3,O15527,HGNC:8125,OGG1,1805373,Arg229Gln,Arg,229,Gln,missense,uniprot
4,O15527,HGNC:8125,OGG1,1801128,Ser320Thr,Ser,320,Thr,missense,uniprot
5,O15527,HGNC:8125,OGG1,3219012,Ala288Val,Ala,288,Val,missense,uniprot
...,...,...,...,...,...,...,...,...,...,...
4082,P04156,HGNC:9449,PRNP,761807915,Asp202Asn,Asp,202,Asn,missense,uniprot
4083,P04156,HGNC:9449,PRNP,776593792,Val203Ile,Val,203,Ile,missense,uniprot
4085,P04156,HGNC:9449,PRNP,751882709,Gln212Pro,Gln,212,Pro,missense,uniprot
4090,P23771,HGNC:4172,GATA3,11567901,Gly242Ser,Gly,242,Ser,missense,uniprot


In [361]:
mutations_uniprot = entradas_solo_uniprot.copy()

In [365]:
mutations_uniprot = mutations_uniprot[['snp_id', 'start_aa', 'from_aa', 'to_aa', 'consequence', 'source']] # formato para tabla db
mutations_uniprot

,snp_id,start_aa,from_aa,to_aa,consequence,source
1,56053615,154,Arg,His,missense,uniprot
2,1052133,326,Ser,Cys,missense,uniprot
3,1805373,229,Arg,Gln,missense,uniprot
4,1801128,320,Ser,Thr,missense,uniprot
5,3219012,288,Ala,Val,missense,uniprot
...,...,...,...,...,...,...
4082,761807915,202,Asp,Asn,missense,uniprot
4083,776593792,203,Val,Ile,missense,uniprot
4085,751882709,212,Gln,Pro,missense,uniprot
4090,11567901,242,Gly,Ser,missense,uniprot


In [386]:
mutations = pd.concat([mutations, mutations_uniprot], ignore_index=True) # Clear the existing index and reset it in the result
mutations

,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,consequence,source
0,587784306,21,46334558.0,46334596.0,156,168.0,HisGln,NaN,deletion,clinvar
1,1555945478,21,46334536.0,46334613.0,148,173.0,ValThr,NaN,deletion,clinvar
2,562568796,21,46435981.0,46435983.0,2944,NaN,Lys,NaN,deletion,clinvar
3,759470656,21,46385936.0,46385938.0,1141,NaN,Leu,NaN,deletion,clinvar
4,398123423,X,77652292.0,77652294.0,1464,NaN,Glu,NaN,deletion,clinvar
...,...,...,...,...,...,...,...,...,...,...
24107,761807915,NaN,NaN,NaN,202,NaN,Asp,Asn,missense,uniprot
24108,776593792,NaN,NaN,NaN,203,NaN,Val,Ile,missense,uniprot
24109,751882709,NaN,NaN,NaN,212,NaN,Gln,Pro,missense,uniprot
24110,11567901,NaN,NaN,NaN,242,NaN,Gly,Ser,missense,uniprot


In [371]:
mutations.start_genomic.astype('Int64')

0       46334558
1       46334536
2       46435981
3       46385936
4       77652292
          ...   
4082        <NA>
4083        <NA>
4085        <NA>
4090        <NA>
4093        <NA>
Name: start_genomic, Length: 24112, dtype: Int64

## agrego disgenet

In [ ]:
disgenet_subset_clinvar[['uniprot', 'hgnc_id','gene_id', 'gene_name', 'snp_id', 'chromosome', 'position', 'consequence', 'source']]

In [403]:
disgenet[['uniprot', 'hgnc_id','gene_id', 'gene_name', 'snp_id', 'chromosome', 'position', 'consequence', 'source']]

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,position,consequence,source
0,O15527,HGNC:8125,4968,OGG1,104893751,3,9750423.0,missense variant,CLINVAR
1,O75494,HGNC:16713,10772,SRSF10,35233638,1,23968097.0,3 prime UTR variant,GWASCAT
2,O75494,HGNC:16713,10772,SRSF10,35233638,1,23968097.0,3 prime UTR variant,GWASCAT
3,O95613,HGNC:16068,5116,PCNT,119479061,21,46346146.0,stop gained,CLINVAR
4,O95613,HGNC:16068,5116,PCNT,119479062,21,46411840.0,stop gained,CLINVAR
...,...,...,...,...,...,...,...,...,...
11544,P26368,HGNC:23156,11338,U2AF2,79546472,19,55662011.0,non coding transcript exon variant,GWASCAT
11545,Q15233,HGNC:7871,4841,NONO,1114167441,X,71298508.0,missense variant,CLINVAR
11546,Q15233,HGNC:7871,4841,NONO,869025343,X,71297938.0,splice region variant,CLINVAR
11547,Q15233,HGNC:7871,4841,NONO,869025345,X,71297900.0,stop gained,CLINVAR


In [401]:
# son mutaciones que estan en clinvar pero no tienen efecto en proteina (Protein change)
# por lo tanto hay que eliminarlas del dataset de disgenet
entradas_disgenet_eliminar

,uniprot,hgnc_id,gene_id,gene_name,snp_id,chromosome,position,consequence,source
12,O95613,HGNC:16068,5116,PCNT,1555954786,21,46353985.0,splice acceptor variant,CLINVAR
14,O95613,HGNC:16068,5116,PCNT,1555959737,21,46363457.0,splice acceptor variant,CLINVAR
18,O95613,HGNC:16068,5116,PCNT,1569178877,21,46347513.0,splice donor variant,CLINVAR
33,O95613,HGNC:16068,5116,PCNT,755084205,21,46388741.0,splice acceptor variant,CLINVAR
52,P38159,HGNC:9910,27316,RBMX,181515589,X,136876663.0,splice region variant,CLINVAR
...,...,...,...,...,...,...,...,...,...
11338,P03372,HGNC:3467,2099,ESR1,1562787582,6,152099207.0,3 prime UTR variant,CLINVAR
11490,P23771,HGNC:4172,2625,GATA3,112417755,10,8073738.0,splice acceptor variant,CLINVAR
11511,P23771,HGNC:4172,2625,GATA3,387906551,10,8064140.0,splice donor variant,CLINVAR
11546,Q15233,HGNC:7871,4841,NONO,869025343,X,71297938.0,splice region variant,CLINVAR


In [395]:
# crear un rango para asignar un id INT a cada mutacion
range(1, len(mutations))

range(1, 24112)

In [17]:
# col id_mutation INT
clinvar['id_mutation'] = range(1, len(clinvar)+1)

In [93]:
mutation.to_csv('datasets/mutations_table.csv', index= False)

In [30]:
mutation

,id_mutation,snp_id,chromosome,start,stop,start_aa,end_aa,from,to,consequence,source
0,1,587784306,21,46334558,46334596,156,168.0,HisGln,NaN,deletion,clinvar
1,2,1555945478,21,46334536,46334613,148,173.0,ValThr,NaN,deletion,clinvar
2,3,562568796,21,46435981,46435983,2944,NaN,Lys,NaN,deletion,clinvar
3,4,759470656,21,46385936,46385938,1141,NaN,Leu,NaN,deletion,clinvar
4,5,398123423,X,77652292,77652294,1464,NaN,Glu,NaN,deletion,clinvar
...,...,...,...,...,...,...,...,...,...,...,...
23121,23122,1057524408,X,71290740,71290740,35,NaN,Gln,Ter,nonsense,clinvar
23122,23123,1085307870,X,71297442,71297442,337,NaN,Arg,Ter,nonsense,clinvar
23123,23124,1555949215,X,71291841,71291841,73,NaN,Arg,Ter,nonsense,clinvar
23124,23125,1602387702,X,71294428,71294428,184,NaN,Arg,Ter,nonsense,clinvar


In [23]:
clinvar.consequence.value_counts()

missense       17953
frameshift      2759
nonsense        1621
deletion         430
insertion        146
duplication      120
delins            97
Name: consequence, dtype: int64

In [24]:
clinvar.type.value_counts()

single nucleotide variant    19351
Deletion                      2028
Duplication                    880
Microsatellite                 424
Indel                          279
Insertion                      146
Inversion                       18
Name: type, dtype: int64